# Data 102 Final Project: Election Finances

By Jessie Houng, Rithika Neti, Ritvik Iyer, and Sunny Shen

*Remark*: The Data Cleaning and EDA sections of the notebook are shown in reverse order compared to where they appear in the report because we performed EDA in multiple stages. 

## 1) Data Cleaning

In [ ]:
# Installing required libraries 
!pip install pandas
!pip install numpy
!pip install fuzzy-match 

In [ ]:
# Import Libraries 
import pandas as pd
import numpy as np
from fuzzy_match import match
import warnings
warnings.simplefilter('ignore')

# Settings
pd.set_option('max_columns', None)

### a) Clean Endorsements Data

In [ ]:
# Read in Primary Candidates 2018 data
dem = pd.read_csv('data/dem_candidates.csv', error_bad_lines=False, encoding='latin-1')
rep = pd.read_csv('data/rep_candidates.csv', error_bad_lines=False, encoding='latin-1')

# Remove candidates not running in national elections 
rep = rep[rep['Office Type'] != 'Governor']
dem = dem[dem['Office Type'] != 'Governor']

# Set column for party affiliations 
dem['Party'] = 'D'
rep['Party'] = 'R'

# Fill values for partisan lean for Republican candidates
partisan_lean_by_district = dict(zip(dem['District'], dem['Partisan Lean']))
rep['Partisan Lean'] = rep['District'].map(partisan_lean_by_district)
prop_filled_v1 = len(rep['Partisan Lean'].dropna())/len(rep['Partisan Lean'])

# Impute partisan lean for Republican candidates based on Senate, then average of House districts within state
senate_leans = dem[dem['District'].str.contains('Senate')][['State', 'Partisan Lean']].drop_duplicates()
partisan_lean_by_state = dict(zip(senate_leans['State'], senate_leans['Partisan Lean']))
house_leans = dem[['Candidate','State','District','Partisan Lean']][
    dem['District'].str.contains('House')].groupby('State').mean().reset_index().dropna()
for state, lean in zip(house_leans['State'], house_leans['Partisan Lean']):
    if state not in partisan_lean_by_state.keys():
        partisan_lean_by_state[state] = lean
for index, row in rep.iterrows():
    state = rep.loc[index, 'State']
    if (pd.isna(rep.loc[index, 'Partisan Lean'])) and (state in partisan_lean_by_state):
        rep.loc[index, 'Partisan Lean'] = partisan_lean_by_state[state]
prop_filled_v2 = len(rep['Partisan Lean'].dropna())/len(rep['Partisan Lean'])

# Identify the states with Republican candidates who still have missing partisan leans 
missing_state_rep = rep[rep['Partisan Lean'].isna()]['State'].value_counts().index.values

# Select the Republican candidates who have missing partisan leans 
invalid_rep_cand = rep[rep['State'].isin(missing_state_rep)]

# Select the Republican candidates who do not have missing partisan leans 
valid_rep_cand = rep[~(rep['State'].isin(missing_state_rep))]

# Deal with missing values in endorsement data
final_dem = dem
final_rep = valid_rep_cand
dem_endorse_support_cols = [x for x in final_dem.columns if 'Support' in x or 'Endorsed' in x]
rep_endorse_support_cols = [x for x in final_rep.columns if 'Support' in x or 'Endorsed' in x]
endorse_map = {'Yes': 1, 'No': -1, np.nan: 0}
final_dem.loc[:, dem_endorse_support_cols] = final_dem.loc[:, dem_endorse_support_cols].replace(endorse_map)
final_rep.loc[:, rep_endorse_support_cols] = final_rep.loc[:, rep_endorse_support_cols].replace(endorse_map)
party_support = np.hstack([final_dem['Party Support?'].values, final_rep['Rep Party Support?'].values])
full_candidates = pd.concat([final_dem, final_rep]).drop(columns=['Party Support?', 'Rep Party Support?'])
full_candidates['Party Support?'] = party_support
cand_endorse_support_cols = [x for x in full_candidates.columns if 'Support' in x or 'Endorsed' in x]
full_candidates.loc[:, cand_endorse_support_cols] = full_candidates.loc[:, cand_endorse_support_cols].fillna(-1)
full_candidates = full_candidates.drop(columns=['General Status', 'Race', 'Veteran?',
                                                'LGBTQ?', 'Elected Official?', 'Self-Funder?',
                                               'STEM?', 'Obama Alum?', 'Guns Sense Candidate?',
                                               'Won Primary'])

# Set district numbers 
full_candidates['District_Num'] = [
    int(x[-1]) if len(x[-1])<=2 else 0 for x in full_candidates['District'].str.split()]

### b) Clean FEC Campaigner Dataset

In [ ]:
# Read in FEC All Candidates data
fin16 = pd.read_csv('data/weball16.txt', sep="|", header=None)
fin18 = pd.read_csv('data/weball18.txt',  sep="|", header=None)

# Set column names  
fin_col = ['CAND_ID', 'CAND_NAME', 'CAND_ICI', 'PTY_CD', 'CAND_PTY_AFFILIATION',
           'TTL_RECEIPTS', 'TRANS_FROM_AUTH', 'TTL_DISB', 'TRANS_TO_AUTH', 'COH_BOP', 
           'COH_COP', 'CAND_CONTRIB', 'CAND_LOANS', 'OTHER_LOANS', 'CAND_LOAN_REPAY', 
           'OTHER_LOAN_REPAY', 'DEBTS_OWED_BY', 'TTL_INDIV_CONTRIB', 'CAND_OFFICE_ST', 
           'CAND_OFFICE_DISTRICT', 'SPEC_ELECTION', 'PRIM_ELECTION', 'RUN_ELECTION', 
           'GEN_ELECTION', 'GEN_ELECTION_PRECENT', 'OTHER_POL_CMTE_CONTRIB',
           'POL_PTY_CONTRIB', 'CVG_END_DT', 'INDIV_REFUNDS', 'CMTE_REFUNDS']
fin16.columns = fin_col
fin18.columns = fin_col

# Set year
fin18['YEAR'] = 18
fin16['YEAR'] = 16

# Combine datasets 
fin = pd.concat([fin16, fin18])

# Reformat party names 
fin['CAND_PTY_AFFILIATION'] = fin['CAND_PTY_AFFILIATION'].map({'REP':'R',
                                                               'DEM':'D'})

# Reformat candidate names  
fin['CAND_NAME'] = [" ".join(n.lower().title().split(", ")[::-1]) for n in fin['CAND_NAME']]

# Remove missing district numbers and re-code data 
fin = fin[~fin['CAND_OFFICE_DISTRICT'].isna()]
fin['CAND_OFFICE_DISTRICT']= fin['CAND_OFFICE_DISTRICT'].astype(int)

### c) Merge Endorsements Data with FEC Campaigner Data

In [ ]:
# Fuzzy match names 
fuzzy_name = [match.extract(x, fin['CAND_NAME'], limit=1, score_cutoff=0.55) for x in full_candidates['Candidate'].values]
fuzzy_name_match = [x[0][0] if x!=[] else '' for x in fuzzy_name]
fuzzy_name_match_score = [x[0][1] if x!=[] else '' for x in fuzzy_name]
full_candidates['fuzzy_name_match'] = fuzzy_name_match
full_candidates['fuzzy_name_match_score'] = fuzzy_name_match_score

# Merge datasets based on fuzzy matched names, state, and party and drop duplicates running in special elections
cand_fin = pd.merge(fin[['CAND_ID', 'CAND_NAME', 'CAND_OFFICE_DISTRICT', 
                         'CAND_OFFICE_ST', 'CAND_ICI', 'PTY_CD', 
                         'CAND_PTY_AFFILIATION', 'TTL_RECEIPTS', 
                        'TRANS_FROM_AUTH','CAND_CONTRIB',
                        'TTL_INDIV_CONTRIB','OTHER_POL_CMTE_CONTRIB']], 
                    full_candidates, 
                    left_on=['CAND_NAME','CAND_OFFICE_ST', 'CAND_PTY_AFFILIATION'] , 
                    right_on=['fuzzy_name_match', 'State', 'Party'])
cand_fin = cand_fin[['Candidate', 'CAND_ID', 'District', 'Party', 'State', 'Emily Endorsed?',
                     'Partisan Lean', 'TTL_RECEIPTS', 'TRANS_FROM_AUTH', 'CAND_CONTRIB',
                     'TTL_INDIV_CONTRIB', 'OTHER_POL_CMTE_CONTRIB', 'Primary %',
                     'Biden Endorsed?', 'Warren Endorsed? ', 'Sanders Endorsed?',
                     'Our Revolution Endorsed?', 'Justice Dems Endorsed?', 
                     'PCCC Endorsed?', 'Indivisible Endorsed?', 'WFP Endorsed?',
                     'VoteVets Endorsed?', 'No Labels Support?', 'Party Support?', 
                     'Trump Endorsed?', 'Bannon Endorsed?', 'Great America Endorsed?',
                     'NRA Endorsed?', 'Right to Life Endorsed?', 'Susan B. Anthony Endorsed?',
                     'Club for Growth Endorsed?', 'Koch Support?', 'House Freedom Support?',
                     'Tea Party Endorsed?', 'Main Street Endorsed?', 'Chamber Endorsed?', 'Race Type', 'CAND_ICI']]

# Find duplicate candidates who are listed as running in multiple roles (i.e incumbent, open race, challenger)
dupes = cand_fin.groupby('Candidate').count()[['CAND_ICI']].reset_index()
dupe_names = dupes[dupes['CAND_ICI'] > 1]['Candidate'].values
keep_dupe_idx = []
for name in dupe_names:
    filtered_dupe_cand = cand_fin[cand_fin['Candidate'] == name]
    argmax_ind_cont = filtered_dupe_cand.TTL_INDIV_CONTRIB.argmax()
    row_idx = filtered_dupe_cand.index[argmax_ind_cont]
    keep_dupe_idx.append(row_idx)
to_remove = cand_fin[cand_fin['Candidate'].isin(dupe_names)].drop(keep_dupe_idx).index.values
cand_fin = cand_fin.drop(to_remove)

### d) Clean FEC Individual Contributions Data

In [ ]:
# Read in Individual Donor data
indiv16 = pd.read_csv('data/indiv16/itcont.txt', sep="|", header=None, low_memory=False)
indiv18 = pd.read_csv('data/indiv18/itcont.txt', sep="|", header=None, low_memory=False)
indiv_cols = pd.read_csv('indiv_header_file.csv').columns

# Filter down to campaign years we are interested in 
P2018_indiv16 = indiv16[indiv16[3]=='P2018']
P2018_indiv18 = indiv18[indiv18[3]=='P2018']

# Set column names 
P2018_indiv16.columns = indiv_cols
P2018_indiv18.columns = indiv_cols

# Concatenate data for 2018 primary cycle together
P2018 = pd.concat([P2018_indiv16, P2018_indiv18])

### e) Clean FEC Campaign-Committee Linkages Data

In [ ]:
# Read in linkage data
ccl16 = pd.read_csv('data/ccl16.txt', sep='|', header=None)
ccl18 = pd.read_csv('data/ccl18.txt', sep='|', header=None)

# Set column names 
ccl_cols = ['CAND_ID', 'CAND_ELECTION_YR', 'FEC_ELECTION_YR', 'CMTE_ID', 'CMTE_TP', 'CMTE_DSGN', 'LINKAGE_ID']
ccl16.columns = ccl_cols
ccl18.columns = ccl_cols

# Filter down to campaign years we are interested in 
ccl16 = ccl16[ccl16['CAND_ELECTION_YR']==2018]
ccl18 = ccl18[ccl18['CAND_ELECTION_YR']==2018]

# Concatenate linkages for 2018 primary cycle together 
ccl = pd.concat([ccl16, ccl18]).drop_duplicates(subset=['CAND_ID', 'CMTE_ID'])

### f) Merge Individual Contributions with Campaign-Committee Linkages

In [ ]:
# Extract Number of Unique individual contributions from 2018 contribution data 
cmt_count = P2018[['CMTE_ID', 'TRANSACTION_AMT', 'NAME']].groupby(
    ['CMTE_ID', 'NAME']).count().reset_index().groupby('CMTE_ID').count().reset_index()
cmt_count = cmt_count.drop(columns=['NAME']).rename(columns={'TRANSACTION_AMT': 'Num_Unique_Contributions'})

# Merge contributions by committee id to linkage data
cand_count = pd.merge(cmt_count, ccl, on = 'CMTE_ID').drop(columns=['CMTE_TP', 'CMTE_DSGN', 'LINKAGE_ID', 
                                                      'CAND_ELECTION_YR', 'FEC_ELECTION_YR']).groupby(
                                                    'CAND_ID').sum().reset_index()

### g) Merge Candidate Endorsements with Number of Unique Donations

## 2) Exploratory Data Analysis

## 3) Method #1: Multiple Hypothesis Testing

## 4) Method #2: Bayesian Inference 